# Extract and Parse

This notebook 

Download an annual report, Extract a particular note to financial statements, Parse it into .md format

In [1]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

#%ls -la /content/drive/MyDrive/docs4Parsing/

## Setup

Install core packages and download relevant files. Here we load some popular ICLR 2024 papers.

In [8]:
#import nest_asyncio

#nest_asyncio.apply()

In [4]:

file_path = "https://raw.githubusercontent.com/tlyim/XtractnParse/main/docs4Parsing/"

papers = [
    "Nextplc2008_segment.pdf",
    "PwC2024_segment.pdf",
#    "deloitte_pp35-39.pdf",
#    "next_pp39-52.pdf",
]

urls = [file_path + filename for filename in papers]

#data_dir = "/content/drive/MyDrive/docs4Parsing"
data_dir = "docs4Parsing"


In [ ]:
# Not working

!mkdir "{data_dir}"
for url, paper in zip(urls, papers):
    !wget "{url}" -O "{data_dir}/{paper}"

#papers
#urls


In [ ]:
# Show the downloaded files

import subprocess
result = subprocess.run(['ls', '-la', data_dir], capture_output=True, text=True, check=True)
print(result.stdout)  # Print the output


# Start from here: 

In [ ]:
#NOT NEED?
# !pip install llama-index


!pip install llama-index-core llama-parse 
#!pip install llama-index-readers-file #python-dotenv

In [2]:
import os

# Access the secrets from environment variables
os.environ["LLAMA_CLOUD_API_KEY"] = os.getenv('LLAMA_CLOUD_API_KEY')

# If running this on Google Colab, you can use the following code to access the secrets
#from google.colab import userdata
#os.environ["LLAMA_CLOUD_API_KEY"] = userdata.get('LLAMA_CLOUD_API_KEY')

# Display environment variables (for debugging purposes)
#print(os.environ.get('LLAMA_CLOUD_API_KEY'))


In [30]:
# bring in our LLAMA_CLOUD_API_KEY
#from dotenv import load_dotenv
#load_dotenv()

# bring in deps
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# set up parser
parser = LlamaParse(#api_key="llx-...", # put your LlamaParse api key here
                    #premium_mode=True,  #Accurate_mode is the default
    parsing_instruction = 
    "The provided pdf document is extracted from the Notes to financial statements in a company annual report. The document contains tables (with financial amounts) and texts from a note on segment reporting. Each page of the pdf document has a footer (with the company name and the year of the report, followed by the page number) and a header (with the section name of the company annual report). Some tables may have table footnotes that are numbered. The table footnotes explain details about certain items in the table that are tagged with superscript numbers. Please convert the pdf document into a markdown document with an equivalent page formatting as similar to the original pdf document as possible. Make sure you convert heading to heading and paragraph to paragraph with exact words and table to table with exact structure of rows and columns.",
    is_formatting_instruction=True,
    disable_ocr=True,
    skip_diagonal_text=True,
    do_not_unroll_columns=False, # unroll columns 
    #target_pages="0,2,7",  # Pages are numbered starting at 0
    bounding_box="0.08,0,0.08,0",
    take_screenshot=True,
    disable_image_extraction=True,
    result_type="markdown"  # "markdown" and "text" are available
)



In [39]:

file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(
    "./docs4Parsing", file_extractor=file_extractor
).load_data()

Started parsing the file under job_id ba255773-b494-411a-974b-33c3f9affb5c
Started parsing the file under job_id 71e202a6-a5d6-4bd4-adcb-bf430fe9fb7f
.Error while parsing the file '<bytes/buffer>': 'markdown'


In [36]:

# use SimpleDirectoryReader to parse our file
file_extractor = {".pdf": parser}
#documents = SimpleDirectoryReader(input_files=['data/canada.pdf'], file_extractor=file_extractor).load_data()
documents = SimpleDirectoryReader(input_files=['docs4Parsing/PwC2024_segment.pdf'], file_extractor=file_extractor).load_data()
#documents = SimpleDirectoryReader(input_files=['docs4Parsing/Nextplc2008_segment.pdf'], file_extractor=file_extractor).load_data()


Started parsing the file under job_id 4ef2d747-ab27-4b45-a54c-4a94cf1ca910


In [40]:
print(documents)

[Document(id_='21a804d6-79b6-43da-b8fb-03f1431c8546', embedding=None, metadata={'file_path': '/workspaces/XtractnParse/docs4Parsing/Nextplc2008_segment.pdf', 'file_name': 'Nextplc2008_segment.pdf', 'file_type': 'application/pdf', 'file_size': 0, 'creation_date': '2025-01-02', 'last_modified_date': '2025-01-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='It seems that there is no content available in the provided text or HTML. If you have specific content from the PDF document that you would like to convert into a markdown format, please provide that text or details, and I will be happy to assist you with the conversion.', mimetype=No

In [41]:
# Custom function to print document details
def print_document(doc):
    doc_dict = vars(doc)
    for key, value in doc_dict.items():
        print(f"{key}: {value}")
    print("\n")

# Print each document in a readable format
for document in documents:
    print_document(document)

id_: 21a804d6-79b6-43da-b8fb-03f1431c8546
embedding: None
metadata: {'file_path': '/workspaces/XtractnParse/docs4Parsing/Nextplc2008_segment.pdf', 'file_name': 'Nextplc2008_segment.pdf', 'file_type': 'application/pdf', 'file_size': 0, 'creation_date': '2025-01-02', 'last_modified_date': '2025-01-02'}
excluded_embed_metadata_keys: ['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date']
excluded_llm_metadata_keys: ['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date']
relationships: {}
metadata_template: {key}: {value}
metadata_separator: 

text_resource: embeddings=None data=None text='It seems that there is no content available in the provided text or HTML. If you have specific content from the PDF document that you would like to convert into a markdown format, please provide that text or details, and I will be happy to assist you with the conversion.' mimetype=None path=None url=None
image_resou

# Section ended here

In [25]:
#import os

# Access the secrets from environment variables
#os.environ["LLAMA_CLOUD_API_KEY"] = os.getenv('LLAMA_CLOUD_API_KEY')
#os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')
#os.environ["HF_TOKEN"] = os.getenv('HF_TOKEN')   # HUGGING FACE

# If running this on Google Colab, you can use the following code to access the secrets
#from google.colab import userdata
#os.environ["LLAMA_CLOUD_API_KEY"] = userdata.get('LLAMA_CLOUD_API_KEY')
#os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
#os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')   # HUGGING FACE

# Display environment variables (for debugging purposes)
#print(os.environ.get('LLAMA_CLOUD_API_KEY'))


In [26]:

# Get API key interactively from user input

#import getpass

#os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass()
#os.environ["GROQ_API_KEY"] = getpass.getpass()
#os.environ["HF_TOKEN"] = getpass.getpass()

#### Parse Documents with **LlamaParse**

In [27]:
from llama_parse import LlamaParse

parser = LlamaParse(#api_key="llx-...", # put your LlamaParse api key here
                    #premium_mode=True,
                    result_type="markdown")

In [28]:
parser

LlamaParse(is_remote=False, api_key='llx-JXE5225hTIy4YHmcYsSXGYzpSiN0zA661gpY9dA1FP2BYYa1', base_url='https://api.cloud.llamaindex.ai', check_interval=1, custom_client=None, ignore_errors=True, max_timeout=2000, num_workers=4, result_type=<ResultType.MD: 'markdown'>, show_progress=True, split_by_page=True, verbose=True, annotate_links=False, auto_mode=False, auto_mode_trigger_on_image_in_page=False, auto_mode_trigger_on_table_in_page=False, auto_mode_trigger_on_text_in_page=None, auto_mode_trigger_on_regexp_in_page=None, azure_openai_api_version=None, azure_openai_deployment_name=None, azure_openai_endpoint=None, azure_openai_key=None, bbox_bottom=None, bbox_left=None, bbox_right=None, bbox_top=None, continuous_mode=False, disable_ocr=False, disable_image_extraction=False, do_not_cache=False, do_not_unroll_columns=False, extract_charts=False, extract_layout=False, fast_mode=False, guess_xlsx_sheet_names=False, html_make_all_elements_visible=False, html_remove_fixed_elements=False, html

In [29]:
from pathlib import Path

paper_dicts = {}

for paper_path in papers:
    paper_base = Path(paper_path).stem
    full_paper_path = str(Path(data_dir) / paper_path)
    md_json_objs = parser.get_json_result(full_paper_path)
    json_dicts = md_json_objs[0]["pages"]
    paper_dicts[paper_path] = {
        "paper_path": full_paper_path,
        "json_dicts": json_dicts,
    }

NameError: name 'papers' is not defined

#### Get Text Nodes

Convert the dictionary above into TextNode objects that we can put into a vector store.

In [53]:
from llama_index.core.schema import TextNode
from typing import Optional

In [54]:
# NOTE: these are utility functions to sort the dumped images by the page number
# (they are formatted like "{uuid}-{page_num}.jpg"
import re


def get_page_number(file_name):
    match = re.search(r"-page-(\d+)\.jpg$", str(file_name))
    if match:
        return int(match.group(1))
    return 0


def _get_sorted_image_files(image_dir):
    """Get image files sorted by page."""
    raw_files = [f for f in list(Path(image_dir).iterdir()) if f.is_file()]
    sorted_files = sorted(raw_files, key=get_page_number)
    return sorted_files

In [55]:
from copy import deepcopy
from pathlib import Path


# attach image metadata to the text nodes
def get_text_nodes(json_dicts, paper_path):
    """Split docs into nodes, by separator."""
    nodes = []

    md_texts = [d["md"] for d in json_dicts]

    for idx, md_text in enumerate(md_texts):
        chunk_metadata = {
            "page_num": idx + 1,
            "paper_path": paper_path,
        }
        node = TextNode(
            text=md_text,
            metadata=chunk_metadata,
        )
        nodes.append(node)

    return nodes

In [56]:
# this will combine all nodes from all papers into a single list
all_text_nodes = []
text_nodes_dict = {}
for paper_path, paper_dict in paper_dicts.items():
    json_dicts = paper_dict["json_dicts"]
    text_nodes = get_text_nodes(json_dicts, paper_dict["paper_path"])
    all_text_nodes.extend(text_nodes)
    text_nodes_dict[paper_path] = text_nodes

#### **AddedByMe:**

Checking the structure of the text_nodes_dict dictionary indexed by the pdf filename


In [ ]:
#print(all_text_nodes)
#print(text_nodes_dict)

#type(text_nodes_dict)

# Number of keys
num_keys = len(text_nodes_dict)
print(f"Number of keys: {num_keys}")

# Lengths of vectors
for key, value in text_nodes_dict.items():
    print(f"Length of vector for key '{key}': {len(value)}")

# Print the strings in each node of a document row by row
for key, value in text_nodes_dict.items():
    print(f"{key}:")
    for item in value:
        print(f"  {item}")

